In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import numpy as np
import os
import time
from torch.utils.data import random_split
from torch.optim import lr_scheduler
from tqdm import tqdm

# class to read dataset with tuples of image and its name (for eval&test in a later stage)
class MYDATA_labeled(Dataset):

    def __init__(self, root, label_csv, transform=None):
        self.transform = transform
        self.target_transform = None
        df = pd.read_csv(label_csv)
        self.img_path_list = []
        self.target_list = []
        for i in range(df.shape[0]):
            self.img_path_list.append(os.path.join(root, df.iloc[i,0]))
            self.target_list.append(df.iloc[i,1]-1)

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = Image.open(self.img_path_list[index]).convert("RGB"), self.target_list[index]

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, self.img_path_list[index]
    
    def __len__(self):
        return len(self.img_path_list)

class MYDATA_unlabeled(Dataset):

    def __init__(self, root, transform=None):
        self.transform = transform
        self.target_transform = None
        self.img_path_list = [os.path.join(root, imgname) for imgname in os.listdir(root)]

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = Image.open(self.img_path_list[index]).convert("RGB"), -1

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, self.img_path_list[index]
    
    def __len__(self):
        return len(self.img_path_list)

In [3]:
transform = {
    "train":transforms.Compose([
        # transforms.RandomCrop((224, 224), pad_if_needed=True), 
        transforms.Resize((224, 224)), 
        transforms.RandomHorizontalFlip(p = 0.5),
        transforms.RandomRotation(10),
        # transforms.RandomVerticalFlip(p=0.5),
        # transforms.ColorJitter(brightness=[0,0.5], contrast=[0,0.5], saturation=[0,0.5], hue=[0,0.5]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4355, 0.4345, 0.4332], std=[0.2830, 0.2826, 0.2818])
    ]),
"val":transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4355, 0.4345, 0.4332], std=[0.2830, 0.2826, 0.2818])
    ])
}

In [6]:
# Load labeled dataset
root_dir = "."
model = models.resnet50(pretrained=True)
feature = model.fc.in_features
model.fc = nn.Linear(in_features=feature,out_features=8,bias=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
val_dir = os.path.join(root_dir, "testset_4k5")
test_label_csv = os.path.join(root_dir, "testset_4k5_true_label_csv.csv")
val_dataset = MYDATA_labeled(val_dir, test_label_csv, transform=transform["val"])
batch_size = 1
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [20]:
basedf = pd.read_csv(test_label_csv).iloc[:,:2]

## pre result of 1k label

In [35]:
label_1k_df = basedf.copy()
# load the last checkpoint with the best model
pretrained_model = os.path.join(root_dir, "checkpoint", "run2", "res_best_acc_checkpoint.pt")
model.load_state_dict(torch.load(pretrained_model))
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [36]:
train_1k_dict = {}
for inputs, labels, imgpath in tqdm(val_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    imgname = imgpath[0].split(os.sep)[-1]
    train_1k_dict[imgname] = int(preds[0].item())
train_1k_pre = []
colname = "base_ypre"
for i in range(basedf.shape[0]):
    train_1k_pre.append(train_1k_dict[basedf.iloc[i,0]]+1)
label_1k_df[colname] = train_1k_pre
acc = ((label_1k_df["true_cat"] == label_1k_df[colname]).sum())/label_1k_df.shape[0]
print(acc)

100%|██████████| 4500/4500 [00:48<00:00, 92.02it/s] 


0.4451111111111111


In [37]:
pretrained_model = os.path.join(root_dir, "checkpoint", "focalloss", "run11", "res_best_acc_checkpoint.pt")
model.load_state_dict(torch.load(pretrained_model))
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [38]:
train_1k_dict = {}
for inputs, labels, imgpath in tqdm(val_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    imgname = imgpath[0].split(os.sep)[-1]
    train_1k_dict[imgname] = int(preds[0].item())
train_1k_pre = []
colname = "focalloss_ypre"
for i in range(basedf.shape[0]):
    train_1k_pre.append(train_1k_dict[basedf.iloc[i,0]]+1)
label_1k_df[colname] = train_1k_pre
acc = ((label_1k_df["true_cat"] == label_1k_df[colname]).sum())/label_1k_df.shape[0]
print(acc)

100%|██████████| 4500/4500 [00:49<00:00, 91.49it/s] 


0.4371111111111111


In [52]:
pretrained_model = os.path.join(root_dir, "checkpoint", "mixlabel", "run11", "res_best_acc_checkpoint.pt")
model.load_state_dict(torch.load(pretrained_model))
model = model.to(device)
model.eval()
train_1k_dict = {}
for inputs, labels, imgpath in tqdm(val_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    imgname = imgpath[0].split(os.sep)[-1]
    train_1k_dict[imgname] = int(preds[0].item())
train_1k_pre = []
colname = "mixlabel_ypre"
for i in range(basedf.shape[0]):
    train_1k_pre.append(train_1k_dict[basedf.iloc[i,0]]+1)
label_1k_df[colname] = train_1k_pre
acc = ((label_1k_df["true_cat"] == label_1k_df[colname]).sum())/label_1k_df.shape[0]
print(acc)

100%|██████████| 4500/4500 [00:54<00:00, 82.20it/s]


0.4577777777777778


In [53]:
label_1k_df.to_csv("1k_resual.csv", index=None)

## pre result of 1k6 label

In [44]:
label_1k6_df = basedf.copy()
# load the last checkpoint with the best model
pretrained_model = os.path.join(root_dir, "checkpoint", "run3", "res_best_acc_checkpoint.pt")
model.load_state_dict(torch.load(pretrained_model))
model = model.to(device)
model.eval()
train_1k6_dict = {}
for inputs, labels, imgpath in tqdm(val_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    imgname = imgpath[0].split(os.sep)[-1]
    train_1k6_dict[imgname] = int(preds[0].item())
train_1k6_pre = []
colname = "base_ypre"
for i in range(basedf.shape[0]):
    train_1k6_pre.append(train_1k6_dict[basedf.iloc[i,0]]+1)
label_1k6_df[colname] = train_1k6_pre
acc = ((label_1k6_df["true_cat"] == label_1k6_df[colname]).sum())/label_1k6_df.shape[0]
print(acc)

100%|██████████| 4500/4500 [00:56<00:00, 79.32it/s] 


0.44155555555555553


In [45]:
# load the last checkpoint with the best model
pretrained_model = os.path.join(root_dir, "checkpoint", "run6", "res_best_acc_checkpoint.pt")
model.load_state_dict(torch.load(pretrained_model))
model = model.to(device)
model.eval()
train_1k6_dict = {}
for inputs, labels, imgpath in tqdm(val_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    imgname = imgpath[0].split(os.sep)[-1]
    train_1k6_dict[imgname] = int(preds[0].item())
train_1k6_pre = []
colname = "focalloss_ypre"
for i in range(basedf.shape[0]):
    train_1k6_pre.append(train_1k6_dict[basedf.iloc[i,0]]+1)
label_1k6_df[colname] = train_1k6_pre
acc = ((label_1k6_df["true_cat"] == label_1k6_df[colname]).sum())/label_1k6_df.shape[0]
print(acc)

100%|██████████| 4500/4500 [00:56<00:00, 80.04it/s]


0.4497777777777778


In [46]:
# load the last checkpoint with the best model
pretrained_model = os.path.join(root_dir, "checkpoint", "run7", "res_best_acc_checkpoint.pt")
model.load_state_dict(torch.load(pretrained_model))
model = model.to(device)
model.eval()
train_1k6_dict = {}
for inputs, labels, imgpath in tqdm(val_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    imgname = imgpath[0].split(os.sep)[-1]
    train_1k6_dict[imgname] = int(preds[0].item())
train_1k6_pre = []
colname = "mixlabel_ypre"
for i in range(basedf.shape[0]):
    train_1k6_pre.append(train_1k6_dict[basedf.iloc[i,0]]+1)
label_1k6_df[colname] = train_1k6_pre
acc = ((label_1k6_df["true_cat"] == label_1k6_df[colname]).sum())/label_1k6_df.shape[0]
print(acc)

100%|██████████| 4500/4500 [00:57<00:00, 78.94it/s]

0.47


In [47]:
label_1k_df.to_csv("1k6_resual.csv", index=None)

## pre result of 3k label

In [48]:
label_3k_df = basedf.copy()
# load the last checkpoint with the best model
pretrained_model = os.path.join(root_dir, "checkpoint", "run5", "res_best_acc_checkpoint.pt")
model.load_state_dict(torch.load(pretrained_model))
model = model.to(device)
model.eval()
train_3k_dict = {}
for inputs, labels, imgpath in tqdm(val_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    imgname = imgpath[0].split(os.sep)[-1]
    train_3k_dict[imgname] = int(preds[0].item())
train_3k_pre = []
colname = "base_ypre"
for i in range(basedf.shape[0]):
    train_3k_pre.append(train_3k_dict[basedf.iloc[i,0]]+1)
label_3k_df[colname] = train_3k_pre
acc = ((label_3k_df["true_cat"] == label_3k_df[colname]).sum())/label_3k_df.shape[0]
print(acc)

100%|██████████| 4500/4500 [00:56<00:00, 79.47it/s]


0.4637777777777778


In [49]:
# load the last checkpoint with the best model
pretrained_model = os.path.join(root_dir, "checkpoint", "focalloss", "run9", "res_best_acc_checkpoint.pt")
model.load_state_dict(torch.load(pretrained_model))
model = model.to(device)
model.eval()
train_3k_dict = {}
for inputs, labels, imgpath in tqdm(val_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    imgname = imgpath[0].split(os.sep)[-1]
    train_3k_dict[imgname] = int(preds[0].item())
train_3k_pre = []
colname = "focalloss_ypre"
for i in range(basedf.shape[0]):
    train_3k_pre.append(train_3k_dict[basedf.iloc[i,0]]+1)
label_3k_df[colname] = train_3k_pre
acc = ((label_3k_df["true_cat"] == label_3k_df[colname]).sum())/label_3k_df.shape[0]
print(acc)

100%|██████████| 4500/4500 [00:57<00:00, 78.92it/s]

0.5008888888888889


In [50]:
# load the last checkpoint with the best model
pretrained_model = os.path.join(root_dir, "checkpoint", "run8", "res_best_acc_checkpoint.pt")
model.load_state_dict(torch.load(pretrained_model))
model = model.to(device)
model.eval()
train_3k_dict = {}
for inputs, labels, imgpath in tqdm(val_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    imgname = imgpath[0].split(os.sep)[-1]
    train_3k_dict[imgname] = int(preds[0].item())
train_3k_pre = []
colname = "mixlabel_ypre"
for i in range(basedf.shape[0]):
    train_3k_pre.append(train_3k_dict[basedf.iloc[i,0]]+1)
label_3k_df[colname] = train_3k_pre
acc = ((label_3k_df["true_cat"] == label_3k_df[colname]).sum())/label_3k_df.shape[0]
print(acc)

100%|██████████| 4500/4500 [00:55<00:00, 80.70it/s] 

0.504


In [51]:
label_1k_df.to_csv("3k_resual.csv", index=None)